In [73]:
from quickstats.components import ExtendedModel
import atlasplots as aplt
import numpy as np
import ROOT
model = ExtendedModel("/afs/ihep.ac.cn/users/c/cyz/besfs/HH_comb/hh_combination_fw_new/run_heft_final/output/workspace/heft/bbbb/0_fit.root", data_name="combData",verbosity="ERROR")
pdf = model.pdf
data = model.data
nuis = model.get_constrained_nuisance_parameters()
cat_map = model.get_category_map()
def get_roorealvar_values(var):
    min_val = var.getMin()
    max_val = var.getMax()
    num_bins = var.getBins()
    bin_width = (max_val - min_val) / num_bins
    values = [min_val + (i + 0.5) * bin_width for i in range(num_bins)]
    return values

In [15]:
all_constraints = model.get_all_constraints()
model.save_snapshot("prefit_nuis",nuis)
model.save_snapshot("prefit_glob",model.global_observables)
output_path = "bbbb_prefit.root"
fo = ROOT.TFile(output_path,"RECREATE")

for cat in cat_map:
    cat_pdf = model.workspace.pdf(cat+"_model")
    if len(cat_map[cat]["observables"].keys())==1:
        observable = model.workspace.obj(list(cat_map[cat]["observables"].keys())[0])
    else:
        print("More than one observable in category, not supported yet")
        continue
    hist_pdf = ROOT.TH1F(cat, cat, observable.getBins(), observable.getMin(), observable.getMax())
    hist_pdf_syst_up,hist_pdf_syst_down = {},{}
    for nui in model.nuisance_parameters:
        nui_name = nui.GetName()
        if 'gamma_' in nui_name and cat not in nui_name: continue
        hist_pdf_syst_up[nui_name] = ROOT.TH1F(cat+"_"+nui_name+"_up", cat+"_"+nui_name+"_up", observable.getBins(), observable.getMin(), observable.getMax())
        hist_pdf_syst_down[nui_name] = ROOT.TH1F(cat+"_"+nui_name+"_down", cat+"_"+nui_name+"_down", observable.getBins(), observable.getMin(), observable.getMax())
    xs = get_roorealvar_values(observable)
    for i,x in enumerate(xs):
        observable.setVal(x)
        hist_pdf.SetBinContent(i+1, cat_pdf.getVal())
    for nui in model.nuisance_parameters:
        nui_name = nui.GetName()
        if nui_name not in hist_pdf_syst_up.keys(): continue
        if 'gamma_' in nui_name:
            prefit_variation, _, nuis_nom = model.inspect_constrained_nuisance_parameter(nui, all_constraints)
        else:
            prefit_variation = 1
            nuis_nom = 0
        nui.setVal(nuis_nom+prefit_variation)
        for i,x in enumerate(xs):
            observable.setVal(x)
            hist_pdf_syst_up[nui_name].SetBinContent(i+1, cat_pdf.getValV())
        nui.setVal(nuis_nom-prefit_variation)
        for i,x in enumerate(xs):
            observable.setVal(x)
            hist_pdf_syst_down[nui_name].SetBinContent(i+1, cat_pdf.getValV())
        nui.setVal(nuis_nom)

    fo.cd()
    hist_pdf.Write()
    for nui in hist_pdf_syst_up:
        hist_pdf_syst_up[nui].Write()
        hist_pdf_syst_down[nui].Write()

fo.Close()
print("Save to",output_path)

Save to bbbb_prefit.root


In [19]:
model.load_snapshot("chhh_fit")

output_path = "bbbb_postfit.root"
fo = ROOT.TFile(output_path,"RECREATE")

for cat in cat_map:
    cat_pdf = model.workspace.pdf(cat+"_model")
    if len(cat_map[cat]["observables"].keys())==1:
        observable = model.workspace.obj(list(cat_map[cat]["observables"].keys())[0])
    else:
        print("More than one observable in category, not supported yet")
        continue
    hist_pdf = ROOT.TH1F(cat, cat, observable.getBins(), observable.getMin(), observable.getMax())
    hist_pdf_syst_up,hist_pdf_syst_down = {},{}
    for nui in model.nuisance_parameters:
        nui_name = nui.GetName()
        if 'gamma_' in nui_name and cat not in nui_name: continue
        hist_pdf_syst_up[nui_name] = ROOT.TH1F(cat+"_"+nui_name+"_up", cat+"_"+nui_name+"_up", observable.getBins(), observable.getMin(), observable.getMax())
        hist_pdf_syst_down[nui_name] = ROOT.TH1F(cat+"_"+nui_name+"_down", cat+"_"+nui_name+"_down", observable.getBins(), observable.getMin(), observable.getMax())
    xs = get_roorealvar_values(observable)
    for i,x in enumerate(xs):
        observable.setVal(x)
        hist_pdf.SetBinContent(i+1, cat_pdf.getVal())
    for nui in model.nuisance_parameters:
        nui_name = nui.GetName()
        if nui_name not in hist_pdf_syst_up.keys(): continue
        nuis_nom = nui.getVal()
        postfit_variation = nui.getError()
        print(cat,nui_name,nuis_nom,postfit_variation)
        nui.setVal(nuis_nom+postfit_variation)
        for i,x in enumerate(xs):
            observable.setVal(x)
            hist_pdf_syst_up[nui_name].SetBinContent(i+1, cat_pdf.getValV())
        nui.setVal(nuis_nom-postfit_variation)
        for i,x in enumerate(xs):
            observable.setVal(x)
            hist_pdf_syst_down[nui_name].SetBinContent(i+1, cat_pdf.getValV())
        nui.setVal(nuis_nom)
        
    fo.cd()
    hist_pdf.Write()
    for nui in hist_pdf_syst_up:
        hist_pdf_syst_up[nui].Write()
        hist_pdf_syst_down[nui].Write()

fo.Close()
print("Save to",output_path)

ggf_16_dEta_1_Xhh_1 alpha_ATLAS_LUMI_Run2 0.004567525212783559 0.9917644863989754
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_E_ggf_16 2.168539533515497 0.6393014883246642
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_E_ggf_17 0.497043499554196 0.6882252143417914
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_E_ggf_18 0.25524582956042013 0.6688614977918013
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_N_ggf_16 -0.44872927520638145 0.7275321766705052
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_N_ggf_17 -1.1801691137968153 0.5845741671674937
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_N_ggf_18 -1.2579905172800019 0.5543985845383885
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_S_ggf_16 1.8688634907203252 0.7184404278731231
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_S_ggf_17 0.5707631751216246 0.6362628351894568
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_S_ggf_18 0.477832857918434 0.8506288349618056
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_W_ggf_16 0.09445340939062155 0.7414250030312819
ggf_16_dEta_1_Xhh_1 alpha_CR12_shape_W_ggf_17 -0.09077454768037152 0.507729

In [5]:
collected_distributions = model.get_collected_distributions(True,[data.GetName()],snapshots=['chhh_fit'])
fo = ROOT.TFile("bbbb_data.root","RECREATE")
for cat in collected_distributions:
    if len(cat_map[cat]["observables"].keys())==1:
        observable = model.workspace.obj(list(cat_map[cat]["observables"].keys())[0])
    else:
        print("More than one observable in category, not supported yet")
        continue
    for case in collected_distributions[cat]:
        hist = ROOT.TH1F(cat+"_"+case, cat+"_"+case, observable.getBins(), observable.getMin(), observable.getMax())
        for i,x in enumerate(collected_distributions[cat][case]['x']):
            hist.SetBinContent(i+1, collected_distributions[cat][case]['y'][i])
            hist.SetBinError(i+1, np.sqrt(collected_distributions[cat][case]['y'][i]))
        fo.cd()
        hist.Write()
fo.Close()

[#1] INFO:Fitting -- using CPU computation library compiled with -mavx2


# Pre-fit

In [48]:
aplt.set_atlas_style()
f_d = ROOT.TFile("bbbb_data.root","READ")
f_a = ROOT.TFile("bbbb_prefit.root","READ")
for cat in collected_distributions:
    h_d = f_d.Get(f"{cat}_combData")
    h_aus = {}
    for h in f_a.GetListOfKeys():
        hname = h.GetName()
        if cat not in hname: continue
        if 'up' in hname:
            h_aus[hname] = f_a.Get(hname)
        elif 'down' in hname:
            continue
        else:
            h_a = f_a.Get(hname)
    x,y,ye_error,xe_error,ye_error_ratio,y_ratio = [],[],[],[],[],[]
    chi2 = 0
    for i in range(h_a.GetNbinsX()):
        error2 = 0
        for j in h_aus:
            error2 += (h_aus[j].GetBinContent(i+1)-h_a.GetBinContent(i+1))**2
        x.append(h_a.GetBinCenter(i+1))
        y.append(h_a.GetBinContent(i+1))
        ye_error.append(np.sqrt(error2))
        xe_error.append(h_a.GetBinWidth(i+1)/2)
        ye_error_ratio.append(ye_error[-1]/y[-1])
        y_ratio.append(1.)
        chi2 +=  (y[-1] - h_d.GetBinContent(i+1))**2/(error2+h_d.GetBinContent(i+1))
    hg = ROOT.TGraphAsymmErrors(len(x),np.array(x),np.array(y),np.array(xe_error),np.array(xe_error),np.array(ye_error),np.array(ye_error))
    hg_ratio = ROOT.TGraphAsymmErrors(len(x),np.array(x),np.array(y_ratio),np.array(xe_error),np.array(xe_error),np.array(ye_error_ratio),np.array(ye_error_ratio))
    fig, (ax1, ax2) = aplt.ratio_plot(name=cat, figsize=(800, 800), hspace=0.05)
    ax1.plot(h_d, "EP X0", label="Data", labelfmt="EP")
    ax1.plot(h_a, linecolor=ROOT.kRed+1, label="Pre-fit S+B", labelfmt="L")
    ax1.plot(hg, "2", fillcolor=1, fillstyle=3254, linewidth=0)
    h_d_ratio = h_d.Clone()
    h_d_ratio.Divide(h_a)
    line = ROOT.TLine(ax1.get_xlim()[0], 1, ax1.get_xlim()[1], 1)
    ax2.plot(line)
    ax2.plot(h_d_ratio, "EP X0")
    ax2.plot(hg_ratio, "2", fillcolor=1, fillstyle=3254, linewidth=0)
    ax2.set_ylim(0.5, 2.0)
    ax2.set_ylabel("Ratio to MC")
    ax2.text(0.6, 0.1, cat, size=28)
    ax1.set_ylabel("Events")
    ax1.add_margins(top=0.15)
    ax1.legend(loc=(0.65, 0.75, 0.92, 0.90))
    ax1.text(0.6, 0.65, f"Chi2 = {chi2:.2f}, ndf = {h_a.GetNbinsX()}", size=26)
    fig.savefig(f"prefit_bbbb/{cat}.png")

Applying ATLAS style settings


Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_1_Xhh_1.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_1_Xhh_2.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_2_Xhh_1.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_2_Xhh_2.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_3_Xhh_1.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_16_dEta_3_Xhh_2.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_17_dEta_1_Xhh_1.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_17_dEta_1_Xhh_2.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_17_dEta_2_Xhh_1.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_17_dEta_2_Xhh_2.png has been created
Info in <TCanvas::Print>: png file prefit_bbbb/ggf_17_dEta_3_Xhh_1.png has been created
Info in <TCanvas::Print>: png fi

# Post-fit

In [60]:
aplt.set_atlas_style()
f_d = ROOT.TFile("bbbb_data.root","READ")
f_a = ROOT.TFile("bbbb_postfit.root","READ")
f_c = ROOT.TFile("/afs/ihep.ac.cn/users/c/cyz/besfs/HH_comb/hh_combination_fw_new/run_heft_final/output/workspace/heft/bbbb/chhh_fit.root","READ")
correlation = f_c.Get("correlation_matrix")
for cat in collected_distributions:
    h_d = f_d.Get(f"{cat}_combData")
    h_aus = {}
    for h in f_a.GetListOfKeys():
        hname = h.GetName()
        if cat not in hname: continue
        if 'up' in hname:
            h_aus[hname] = f_a.Get(hname)
        elif 'down' in hname:
            continue
        else:
            h_a = f_a.Get(hname)
    x,y,ye_error,xe_error,ye_error_ratio,y_ratio = [],[],[],[],[],[]
    nbin = h_a.GetNbinsX()
    for i in range(nbin):
        error2 = 0
        for n in h_aus:
            error2 += (h_aus[n].GetBinContent(i+1)-h_a.GetBinContent(i+1))**2
        x.append(h_a.GetBinCenter(i+1))
        y.append(h_a.GetBinContent(i+1))
        ye_error.append(np.sqrt(error2))
        xe_error.append(h_a.GetBinWidth(i+1)/2)
        ye_error_ratio.append(ye_error[-1]/y[-1])
        y_ratio.append(1.)
    c = np.zeros((nbin, nbin))
    for i in range(nbin):
        ynom_i = y[i]
        for j in range(i, nbin):
            sumbin = 0
            ynom_j = y[j]
            for n in h_aus:
                ysyst_i_n = h_aus[n].GetBinContent(i+1)
                for m in h_aus:
                    ysyst_j_m = h_aus[m].GetBinContent(j+1)
                    if n == m:
                        corr = 1.
                    else:
                        n_bin = correlation.GetXaxis().FindBin(n)
                        m_bin = correlation.GetYaxis().FindBin(m)
                        corr = correlation.GetBinContent(n_bin, m_bin)
                    sumbin += (ysyst_i_n - ynom_i) * corr * (ysyst_j_m - ynom_j)
            if i == j:
                sumbin += ynom_i
            c[i, j] = sumbin
            if i != j:
                c[j, i] = sumbin
    c = np.linalg.inv(c)
    chi2 = 0
    for i in range(nbin):
        for j in range(nbin):
            chi2 += (y[i] - h_d.GetBinContent(i+1)) * c[i, j] * (y[j] - h_d.GetBinContent(j+1))
    hg = ROOT.TGraphAsymmErrors(len(x),np.array(x),np.array(y),np.array(xe_error),np.array(xe_error),np.array(ye_error),np.array(ye_error))
    hg_ratio = ROOT.TGraphAsymmErrors(len(x),np.array(x),np.array(y_ratio),np.array(xe_error),np.array(xe_error),np.array(ye_error_ratio),np.array(ye_error_ratio))
    fig, (ax1, ax2) = aplt.ratio_plot(name=cat, figsize=(800, 800), hspace=0.05)
    ax1.plot(h_d, "EP X0", label="Data", labelfmt="EP")
    ax1.plot(h_a, linecolor=ROOT.kRed+1, label="Post-fit S+B", labelfmt="L")
    ax1.plot(hg, "2", fillcolor=1, fillstyle=3254, linewidth=0)
    h_d_ratio = h_d.Clone()
    h_d_ratio.Divide(h_a)
    ax2.plot(h_d_ratio, "EP X0")
    ax2.plot(hg_ratio, "2", fillcolor=1, fillstyle=3254, linewidth=0)
    ax2.set_ylim(0.5, 2.0)
    ax2.set_ylabel("Ratio to MC")
    ax2.text(0.6, 0.1, cat, size=28)
    ax1.set_ylabel("Events")
    ax1.add_margins(top=0.15)
    ax1.legend(loc=(0.65, 0.75, 0.92, 0.90))
    ax1.text(0.6, 0.65, f"Chi2 = {chi2:.2f}, ndf = {nbin}", size=26)
    fig.savefig(f"postfit_bbbb/{cat}.png")


Applying ATLAS style settings


Warning in <TCanvas::Constructor>: Deleting canvas with same name: ggf_16_dEta_1_Xhh_1
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_1_Xhh_1.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_1_Xhh_2.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_2_Xhh_1.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_2_Xhh_2.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_3_Xhh_1.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_16_dEta_3_Xhh_2.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_17_dEta_1_Xhh_1.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_17_dEta_1_Xhh_2.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_17_dEta_2_Xhh_1.png has been created
Info in <TCanvas::Print>: png file postfit_bbbb/ggf_17_dEta_2_Xhh_2.png has been created
Info in <TCanvas::Print